In [15]:
import pandas as pd

# Load the Fear & Greed Index data
try:
    fgi_df = pd.read_csv("fear_greed_index.csv")
    print("--- fear_greed_index.csv Head ---")
    print(fgi_df.head())
    print("\n--- fear_greed_index.csv Info ---")
    fgi_df.info()
except Exception as e:
    print(f"Error loading fear_greed_index.csv: {e}")

# Load the Historical Trader data
try:
    hd_df = pd.read_csv("historical_data.csv")
    print("\n--- historical_data.csv Head ---")
    print(hd_df.head())
    print("\n--- historical_data.csv Info ---")
    hd_df.info()
except Exception as e:
    print(f"Error loading historical_data.csv: {e}")

--- fear_greed_index.csv Head ---
    timestamp  value classification        date
0  1517463000     30           Fear  2018-02-01
1  1517549400     15   Extreme Fear  2018-02-02
2  1517635800     40           Fear  2018-02-03
3  1517722200     24   Extreme Fear  2018-02-04
4  1517808600     11   Extreme Fear  2018-02-05

--- fear_greed_index.csv Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       2644 non-null   int64 
 1   value           2644 non-null   int64 
 2   classification  2644 non-null   object
 3   date            2644 non-null   object
dtypes: int64(2), object(2)
memory usage: 82.8+ KB

--- historical_data.csv Head ---
                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082e

In [16]:
import pandas as pd

# Load the datasets
fgi_df = pd.read_csv("fear_greed_index.csv")
hd_df = pd.read_csv("historical_data.csv")

# --- 1. Data Cleaning and Preparation for Merging ---


fgi_df = fgi_df[['date', 'value', 'classification']].rename(columns={'date': 'Date'})
# Convert 'Date' to datetime
fgi_df['Date'] = pd.to_datetime(fgi_df['Date'])

hd_df['Timestamp IST'] = pd.to_datetime(hd_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
# Extract the date component for daily join
hd_df['Date'] = hd_df['Timestamp IST'].dt.date

hd_df['Date'] = pd.to_datetime(hd_df['Date'])

# --- 2. Merge DataFrames ---
# Left merge: Keep all trade records and add daily sentiment
merged_df = pd.merge(
    hd_df,
    fgi_df,
    on='Date',
    how='left'
)

# --- 3. Initial Check of Merged Data ---
print("--- Merged DataFrame Info (Post-Merge) ---")
merged_df.info()
print("\n--- Merged DataFrame Head (Post-Merge) ---")
print(merged_df[['Date', 'Timestamp IST', 'Closed PnL', 'classification', 'value']].head())
print("\n--- Check for Unmatched Dates (NaNs in sentiment columns) ---")
unmatched_count = merged_df['classification'].isnull().sum()
print(f"Number of trades without matching sentiment data: {unmatched_count} out of {len(merged_df)}")

--- Merged DataFrame Info (Post-Merge) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Account           211224 non-null  object        
 1   Coin              211224 non-null  object        
 2   Execution Price   211224 non-null  float64       
 3   Size Tokens       211224 non-null  float64       
 4   Size USD          211224 non-null  float64       
 5   Side              211224 non-null  object        
 6   Timestamp IST     211224 non-null  datetime64[ns]
 7   Start Position    211224 non-null  float64       
 8   Direction         211224 non-null  object        
 9   Closed PnL        211224 non-null  float64       
 10  Transaction Hash  211224 non-null  object        
 11  Order ID          211224 non-null  int64         
 12  Crossed           211224 non-null  bool          
 13  Fee             

In [17]:
import numpy as np
import pandas as pd
import altair as alt
import matplotlib as mlt

# Filter out the 6 rows with missing sentiment data
analysis_df = merged_df.dropna(subset=['classification', 'value']).copy()

# Define a categorical order for sentiment (from Fear to Greed)
sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
analysis_df['classification'] = pd.Categorical(analysis_df['classification'], categories=sentiment_order, ordered=True)

# 1. Overall Performance by Sentiment
sentiment_performance = analysis_df.groupby('classification').agg(
    Total_PnL=('Closed PnL', 'sum'),
    Average_PnL=('Closed PnL', 'mean'),
    Trade_Count=('Closed PnL', 'count')
).reset_index()

# Print the table
print("--- Sentiment Performance Summary Table ---")
print(sentiment_performance.to_markdown(index=False, floatfmt=".2f"))

# 2. Visualize Total PnL by Sentiment Classification
base = alt.Chart(sentiment_performance).encode(
    x=alt.X('classification', sort=sentiment_order, title='Market Sentiment Classification', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Total_PnL', title='Total Closed PnL (USD)'),
    color=alt.condition(
        alt.datum.Total_PnL > 0,
        alt.value('darkgreen'),  # Green for profit
        alt.value('darkred')    # Red for loss
    ),
    tooltip=['classification', alt.Tooltip('Total_PnL', format=',.2f')]
).properties(
    title='Total Closed PnL by Market Sentiment'
)

chart = base.mark_bar().encode(
    order=alt.Order('classification_sort_index', sort='ascending')
)

# Add text labels on the bars
text = base.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudge text up
).encode(
    text=alt.Text('Total_PnL', format=',.0f'),
    color=alt.value('black') # Black color for text
)

# Save the chart
final_chart = (chart + text).save('total_pnl_by_sentiment.json')

--- Sentiment Performance Summary Table ---
| classification   |   Total_PnL |   Average_PnL |   Trade_Count |
|:-----------------|------------:|--------------:|--------------:|
| Extreme Fear     |   739110.25 |         34.54 |         21400 |
| Fear             |  3357155.44 |         54.29 |         61837 |
| Neutral          |  1292920.68 |         34.31 |         37686 |
| Greed            |  2150129.27 |         42.74 |         50303 |
| Extreme Greed    |  2715171.31 |         67.89 |         39992 |


C:\Users\prafu\AppData\Local\Temp\ipykernel_8336\3525391781.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sentiment_performance = analysis_df.groupby('classification').agg(


ValueError: Unable to determine data type for the field "classification_sort_index"; verify that the field name is not misspelled. If you are referencing a field from a transform, also confirm that the data type is specified correctly.

In [18]:

analysis_df = merged_df.dropna(subset=['classification', 'value']).copy()

# Define a categorical order for sentiment (from Fear to Greed)
sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
analysis_df['classification'] = pd.Categorical(analysis_df['classification'], categories=sentiment_order, ordered=True)

# 1. Overall Performance by Sentiment
sentiment_performance = analysis_df.groupby('classification').agg(
    Total_PnL=('Closed PnL', 'sum'),
    Average_PnL=('Closed PnL', 'mean'),
    Trade_Count=('Closed PnL', 'count')
).reset_index()

# Print the table
print("--- Sentiment Performance Summary Table ---")
print(sentiment_performance.to_markdown(index=False, floatfmt=".2f"))

# --- 2. Visualization Functions (to avoid repetition) ---
def create_sentiment_chart(data, y_col, y_title, chart_title, filename):
    base = alt.Chart(data).encode(
        # Use the explicit order defined earlier for correct sorting
        x=alt.X('classification', sort=sentiment_order, title='Market Sentiment Classification', axis=alt.Axis(labelAngle=-45)),
        y=alt.Y(y_col, title=y_title),
        color=alt.condition(
            alt.datum[y_col] > 0,
            alt.value('darkgreen'),  # Green for profit
            alt.value('darkred')    # Red for loss
        ),
        tooltip=['classification', alt.Tooltip(y_col, format=',.2f')]
    ).properties(
        title=chart_title
    )

    chart = base.mark_bar()

    # Add text labels on the bars
    text = base.mark_text(
        align='center',
        baseline='middle',
        dy=-10  # Nudge text up
    ).encode(
        text=alt.Text(y_col, format=',.0f'),
        color=alt.value('black') # Black color for text
    )

    # Save the chart
    (chart + text).save(filename)
    return filename

# --- 3. Generate Charts ---
# Chart A: Total PnL
create_sentiment_chart(
    sentiment_performance,
    'Total_PnL',
    'Total Closed PnL (USD)',
    'Total Closed PnL by Market Sentiment',
    'total_pnl_by_sentiment.json'
)

# Chart B: Average PnL
create_sentiment_chart(
    sentiment_performance,
    'Average_PnL',
    'Average Closed PnL per Trade (USD)',
    'Average Closed PnL per Trade by Market Sentiment',
    'average_pnl_by_sentiment.json'
)

# --- 4. Performance by Trade Direction within Sentiment ---
direction_sentiment_performance = analysis_df.groupby(['classification', 'Direction']).agg(
    Total_PnL=('Closed PnL', 'sum'),
    Average_PnL=('Closed PnL', 'mean'),
    Trade_Count=('Closed PnL', 'count')
).reset_index()

# Print the table
print("\n--- Sentiment and Direction Performance Summary Table (Top 10) ---")
print(direction_sentiment_performance.sort_values(by='Total_PnL', ascending=False).head(10).to_markdown(index=False, floatfmt=".2f"))

# Visualize Average PnL by Direction and Sentiment
direction_chart = alt.Chart(direction_sentiment_performance).encode(
    x=alt.X('classification', sort=sentiment_order, title='Market Sentiment Classification', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Average_PnL', title='Average PnL (USD)'),
    color=alt.Color('Direction', title='Trade Type'),
    tooltip=['classification', 'Direction', alt.Tooltip('Average_PnL', format=',.3f'), alt.Tooltip('Trade_Count', format=',')]
).properties(
    title='Average PnL by Trade Direction and Market Sentiment'
).mark_bar().save('average_pnl_by_direction_sentiment.json')

C:\Users\prafu\AppData\Local\Temp\ipykernel_8336\3040519376.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sentiment_performance = analysis_df.groupby('classification').agg(
C:\Users\prafu\AppData\Local\Temp\ipykernel_8336\3040519376.py:70: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  direction_sentiment_performance = analysis_df.groupby(['classification', 'Direction']).agg(
c:\Users\prafu\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_

--- Sentiment Performance Summary Table ---
| classification   |   Total_PnL |   Average_PnL |   Trade_Count |
|:-----------------|------------:|--------------:|--------------:|
| Extreme Fear     |   739110.25 |         34.54 |         21400 |
| Fear             |  3357155.44 |         54.29 |         61837 |
| Neutral          |  1292920.68 |         34.31 |         37686 |
| Greed            |  2150129.27 |         42.74 |         50303 |
| Extreme Greed    |  2715171.31 |         67.89 |         39992 |

--- Sentiment and Direction Performance Summary Table (Top 10) ---
| classification   | Direction   |   Total_PnL |   Average_PnL |   Trade_Count |
|:-----------------|:------------|------------:|--------------:|--------------:|
| Extreme Greed    | Sell        |  2081660.35 |        290.57 |          7164 |
| Fear             | Close Short |  1914986.46 |        207.68 |          9221 |
| Fear             | Close Long  |  1432581.92 |         83.00 |         17260 |
| Greed       